# Computer Vision – Gradients & Edges

Utilizamos gradientes para detectar bordes en imágenes, lo que nos permite encontrar contornos y contornos de objetos en imágenes. Se utilizan como entradas para cuantificar imágenes a través de la extracción de características, de hecho, los descriptores de imagen altamente exitosos y conocidos, como el histograma de gradientes orientados(HOG) y SIFT, se basan en las representaciones de gradiente de imagen. 

Los gradientes se utilizan incluso para construir mapas de bordes y contornos, que resaltan los objetos de una imagen. Por lo tanto, utilizamos gradientes todo el tiempo en la visión por computadora y el procesamiento de imágenes.

Como mencioné en la introducción, los gradientes de imagen se usan como bloques de construcción básicos en muchas aplicaciones de procesamiento de imágenes y visión por computadora. Sin embargo, la aplicación principal de los gradientes de imagen se encuentra dentro de la detección de bordes. 

Como sugiere su nombre, la detección de bordes es el proceso de encontrar bordes en una imagen, que revelará información estructural sobre los objetos en una imagen. Por lo tanto, los bordes podrían corresponder a:

1. Límites de un objeto en una imagen.
2. Límites de sombreado o condiciones de iluminación en una imagen.
3. Límites de “partes” dentro de un objeto.


Formalmente, un gradiente de imagen se define como un cambio direccional en la intensidad de la imagen. En la práctica, los gradientes de imagen se estiman utilizando kernels como lo hicimos con el blur(difuminado) y smooth(suavizado), pero esta vez estamos tratando de encontrar los componentes estructurales de la imagen. Nuestro objetivo aquí es encontrar el cambio de dirección hacia el píxel central marcado en rojo tanto en la dirección x como en la dirección y

La **magnitud del gradiente** se utiliza para medir qué tan fuerte es el cambio en la intensidad de la imagen. La magnitud del gradiente es un número de valor real que cuantifica la "fuerza" del cambio en la intensidad. 

Mientras que la **orientación del gradiente** se usa para determinar en qué dirección apunta el cambio de intensidad. Como sugiere su nombre, la orientación del degradado nos dará un ángulo o que podemos usar para cuantificar la dirección del cambio.


### Setup

In [1]:
import numpy as np
import argparse
import cv2

image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/1_Basics_of_Computer_Vision/images/bricks.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Original", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Calculate gradient manually

In [2]:
# Calculo del los gradientes sobre los ejes X y Y respectivamente
gX = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
gY = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

# Computar el gradiente de magnitud y orientación
mag = np.sqrt((gX ** 2) + (gY ** 2))
orientation = np.arctan2(gY, gX) * (180 / np.pi) % 180

# Encuentrar todos los píxeles que están dentro de los límites de ángulo superior e inferior
idxs = np.where(orientation >= 50.0, orientation, -1)
idxs = np.where(orientation <= 80.0, idxs, -1)
mask = np.zeros(gray.shape, dtype="uint8")
mask[idxs > -1] = 255

cv2.imshow("Mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Sobel Kernels to calculate gradients

In [3]:
# Calculo del los gradientes sobre los ejes X y Y respectivamente
gX = cv2.Sobel(gray, ddepth=cv2.CV_64F, dx=1, dy=0)
gY = cv2.Sobel(gray, ddepth=cv2.CV_64F, dx=0, dy=1)

# las imágenes `gX` y` gY` son ahora del tipo de datos de punto flotante,
# por lo que debemos tener cuidado de convertirlos de nuevo a 8 bits sin convertir
# una representación entera para que otras funciones de OpenCV puedan utilizarlas
gX = cv2.convertScaleAbs(gX)
gY = cv2.convertScaleAbs(gY)

# Combina las representaciones sobel X e Y en una sola imagen
sobelCombined = cv2.addWeighted(gX, 0.5, gY, 0.5, 0)

cv2.imshow("Sobel X", gX)
cv2.imshow("Sobel Y", gY)
cv2.imshow("Sobel Combined", sobelCombined)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Edges

El borde o "Edge" se define como discontinuidades en la intensidad de los píxeles, o la aparición de una gran diferencia y cambio en los valores de los píxeles.

### Sted Edge

Un borde escalonado o "Step Edge" se forma cuando hay un cambio abrupto en la intensidad de píxel de un lado de la discontinuidad al otro. o que indica un cambio brusco en el valor de píxel. Estos tipos de bordes tienden a ser fáciles de detectar.

![](images/Imagen1.png)

### Ramp Edge

Un borde de rampa o "Ramp Edge" es como un "Step Edge", solo que el cambio en la intensidad de píxeles no es instantáneo. En cambio, el cambio en el valor de píxel se produce una distancia corta, pero finita.

![](images/Imagen2.png)

### Roof Edge

Un borde de cresta o "Roof Edge" es similar a la combinación de dos "Ramps Edges", uno golpeadose contra el otro. En el contexto de la detección de bordes, se produce un borde de cresta cuando la intensidad de la imagen cambia bruscamente, pero luego vuelve al valor inicial después de una corta distancia.

![](images/Imagen3.png)

### Ridge Edge

A diferencia del "Roof Edge" donde hay una meseta corta y finita en la parte superior del borde, el borde del techo no tiene tal meseta. En su lugar, subimos lentamente a ambos lados del borde, pero la parte superior es un pináculo y simplemente volvemos a caer por la parte inferior.

![](images/Imagen4.png)